# Lazy vs Eager

Understand the difference between eager and lazy graph tracing and transpilation.

⚠️ If you are running this notebook in Colab, you will have to install `Ivy` and some dependencies manually. You can do so by running the cell below ⬇️

If you want to run the notebook locally but don't have Ivy installed just yet, you can check out the [Get Started section of the docs.](https://www.docs.ivy.dev/overview/get_started.html)

In [1]:
!pip install ivy

`ivy.trace_graph` can be performed either eagerly or lazily since it depends on function tracing, which requires function arguments to use for tracing. `ivy.transpile` however doesn't depend on function tracing and so is applied eagerly without requiriing function arguments. One exception though where `ivy.transpile` defaults to lazy transpilation is when transpiling entire modules like `kornia`. It isn't until a function or a class is executed from the lazy transpiled kornia  module that the transpilation is actually performed.

For `ivy.trace_graph`, the arguments can be provided during the  `ivy.trace_graph` call itself, in which case the process is performed **eagerly**. We show some simple examples for each case below.

## Trace

In the example below, the function is traced **lazily**, which means the first function call will execute slowly, as this is when the tracing process actually occurs.

In [1]:
import ivy

def normalize(x):
    mean = ivy.mean(x)
    std = ivy.std(x)
    return ivy.divide(ivy.subtract(x, mean), std)


Following are the supported configurations :
compiler : cp38-cp38-manylinux_2_17_x86_64, cp38-cp38-win_amd64, cp39-cp39-manylinux_2_17_x86_64, cp39-cp39-win_amd64, cp310-cp310-manylinux_2_17_x86_64, cp310-cp310-win_amd64, cp310-cp310-macosx_12_0_arm64, cp311-cp311-manylinux_2_17_x86_64, cp311-cp311-win_amd64, cp311-cp311-macosx_12_0_arm64, cp312-cp312-manylinux_2_17_x86_64, cp312-cp312-win_amd64, cp312-cp312-macosx_12_0_arm64



And let's also create the dummy `numpy` arrays as before:

In [2]:
# import NumPy
import numpy as np
np.random.seed(0)

# create random numpy array for testing
x = np.random.uniform(size=10)

Let's assume that our target framework is `tensorflow`:

In [3]:
import tensorflow as tf
ivy.set_backend("tensorflow")

x = tf.constant(x)

2024-10-19 02:11:29.834162: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 02:11:29.965072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 02:11:30.015897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 02:11:30.028543: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 02:11:30.107819: I tensorflow/core/platform/cpu_feature_guar

In [4]:
norm_trace = ivy.trace_graph(normalize)  # Lazy transpilation as no args are provided, executes quicker

In [5]:
norm_trace(x) # lazy -> eager transpilation as args are provided and the function is called for the first time, executes slowly

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.36293708,  0.53895184, -0.07048605, -0.38424253, -1.04139636,
        0.16331669, -0.96587166,  1.49617496,  1.88587438, -1.25938418])>

In [6]:
norm_trace(x) # fast, traced on previous call

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.36293708,  0.53895184, -0.07048605, -0.38424253, -1.04139636,
        0.16331669, -0.96587166,  1.49617496,  1.88587438, -1.25938418])>

However, in the following example the tracing occurs **eagerly**, and both function calls will be fast:

In [7]:
norm_tracing = ivy.trace_graph(normalize, args=(x,))  # eager transpilation as args are provided, executes slowly

In [8]:
norm_tracing(x) # fast, traced at ivy.trace_graph

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.36293708,  0.53895184, -0.07048605, -0.38424253, -1.04139636,
        0.16331669, -0.96587166,  1.49617496,  1.88587438, -1.25938418])>

In [9]:
norm_tracing(x) # fast, traced at ivy.trace_graph

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.36293708,  0.53895184, -0.07048605, -0.38424253, -1.04139636,
        0.16331669, -0.96587166,  1.49617496,  1.88587438, -1.25938418])>

## Transpile

Let's redefine the `normalize` function in PyTorch and transpile it to `TensorFlow`:

In [10]:
import torch


def normalize(x):
    mean = torch.mean(x)
    std = torch.std(x)
    return torch.div(torch.sub(x, mean), std)

In the example below, the function is transpiled **eagerly** and so this cell executes slower because this is when the transpilation process actually occurs.

In [11]:
norm_trans = ivy.transpile(normalize, source="torch", target="tensorflow")

Transpilation of normalize complete.                                           


In [12]:
norm_trans(x) # fast, transpiled at ivy.transpile

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([-0.34431235,  0.51129461, -0.06686894, -0.36452447, -0.98795534,
        0.15493582, -0.91630631,  1.41939619,  1.78909753, -1.19475674])>

However, in the following example the transpilation occurs *lazily* since we are transpiling a module `kornia` to `jax`, hence the first function call will be slower where the actual transpilation will happen while the subsequent  function calls will be fast:

In [2]:
import kornia

jax_kornia = ivy.transpile(kornia, source="torch", target="jax")

Let's actually call a function from our transpiled kornia which will trigger the transpilation to happen eagerly, resulting in the first invocation to be slow but the subsequent ones to be faster

In [3]:
import jax
import numpy as np

input = jax.numpy.array(np.random.rand(2, 3, 4, 5))
gray = jax_kornia.color.bgr_to_grayscale(input)  # 2x1x4x5

Let's run another function from `jax_kornia` to finally wrap up this demo!

In [8]:
gray = jax_kornia.color.bgr_to_grayscale(input)  # 2x1x4x5
print(type(gray))

<class 'jaxlib.xla_extension.ArrayImpl'>


## Round Up

That's it, you now know the difference between lazy vs eager execution for `ivy.trace_graph` and `ivy.transpile`! Next, we'll be exploring how `ivy.trace_graph`can be called as a function decorator!